# HCP Vault Dedicated Demo for GitHub Actions

## Setup of the Demo

This setup is tested on MacOS. The components used in this demo are:
- HCP Vault Dedicated cluster running
- You have the Vault CLI installed

This assumes your Vault cluster is on the public HVN and you have set your VAULT_ADDR and VAULT_TOKEN variables.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.


# Setup Vault Environment Variables

In [ ]:
# Retrieve the Vault Public Cluster URL from portal.cloud.hashicorp.com
export VAULT_ADDR=""
export VAULT_NAMESPACE="admin"

# Retrieve the Vault admin token from portal.cloud.hashicorp.com
# The token expires after 6 hours. You can refresh it before expiry
export VAULT_TOKEN=""

# Configure Vault

In [ ]:
# Configure a JWT auth mount for the GitHub Actions OIDC provider
# A single JWT auth mount resource may service multiple underlying roles and must only be configured once for a Vault namespace.

vault auth enable \
  -path=jwt-github-actions \
  -description="JWT Auth Backend for GitHub Actions" \
  -default-lease-ttl=30 \
  jwt

# These parameters allow the Vault server to verify the received JSON Web Tokens (JWT) during the authentication process.
vault write auth/jwt-github-actions/config \
  bound_issuer="https://token.actions.githubusercontent.com" \
  oidc_discovery_url="https://token.actions.githubusercontent.com"

# Configure a Vault policy for a GitHub Actions workflow
# This policy grants read access to all secrets under the path `kv/app1`
echo 'path "kv/data/app1" {
  capabilities = ["read"]
}
' | vault policy write app1 -

# Configure a JWT auth role for a GitHub Actions workflow
# Create a Vault JWT auth role for each unique CI workflow
# Ensure that the bound_claims parameter is defined for your security requirements, and has at least one condition. Optionally, you can also set the bound_subject as well as the bound_audiences parameter.
# This auth role may include repository and workflow information like:
## Specific audience, subject
## Repository, branch
## Actor, workflow name
## Repository visibility setting, runner environment, and repository owner.
vault write auth/jwt-github-actions/role/app1 -<<EOF
{
  "role_type": "jwt",
  "bound_audiences": ["https://github.com/janehojy"],
  "bound_claims_type": "string",
  "user_claim": "sub",
  "bound_claims": {
    "repository": "janehojy/vault-demo-gha"
  },
  "policies": ["app1"],
  "ttl": "10m"
}
EOF

In [ ]:
vault kv put kv/app1 password=P@55w0rD!

# Cleanup

In [ ]:
vault kv destroy -versions=1 kv/app1

vault kv metadata delete kv/app1

vault kv get kv/app1

# 